In [4]:
# Import Libraries
import quandl
import pandas_datareader.data as web
import datetime
import yfinance as yf
import pandas as pd
from datetime import datetime
import seaborn as sns
import os
import re
import matplotlib as plt

# Pull S&P 500 and VIX Data from Yahoo
indices_data = yf.download('^GSPC ^VIX', start='1998-12-31', end='2023-11-01')

# Create liquidity, SP500, and Volatility columns
liquidity = pd.DataFrame(indices_data['Volume']['^GSPC'])
SP_500 = pd.DataFrame(indices_data['Close']['^GSPC'])
volatility = pd.DataFrame(indices_data['Close']['^VIX'])

# Pull Michigan University Sentiment Data from FRED
sent_data = web.DataReader(['UMCSENT'], 'fred', datetime(1998, 12, 31), datetime(2023, 11, 13))

# Set my quandl key for pulling data
quandl.ApiConfig.api_key = 'yC7JQzd6FsYXc_4F67ym'

# Create a dictionary of the indicators to pull from quandl
macro_indicators = {
    '2_YR_Treasury': 'FRED/DGS2',
    'Eff_Fed_Funds': 'FRED/DFF',
    'Real_GDP': 'FRED/GDPC1',
    'Core_CPI': 'FRED/CORESTICKM159SFRBATL',
    'PCE': 'FRED/PCE',
    'Unemployment': 'FRED/UNRATE',
    'Savings_Rate': 'FRED/PSAVERT',
    'Retail_Sales': 'FRED/RETAILMPCSMSA',
    'Manufacturing_PMI': 'ISM/MAN_PMI'
}

# Create an empty dictionary of dataframes
macro_indicators_df = {}

# for each indicator
for indicator in macro_indicators:
    
    # Pull the quandl data and add it to the dictionary
    quandl_code = macro_indicators[indicator]
    data = quandl.get(quandl_code)
    macro_indicators_df[indicator] = data

# Add data pulled from Fred and Yahoo to dictionary
macro_indicators_df['Consumer_Sent'] = sent_data
macro_indicators_df['Liquidity'] = liquidity
macro_indicators_df['Volatility'] = volatility
macro_indicators_df['SP_500'] = SP_500

# Merge all of the data into one df
merged = pd.DataFrame(index= pd.date_range(start=datetime(1998,12,31), end=datetime.now(), freq='D'))
for indicator in macro_indicators_df.keys():
    merged = pd.merge(merged, macro_indicators_df[indicator], left_index=True, right_index=True, how='left')

# Impute the empty values with the most recent value, calculate pct change from 1 yr ago
merged.columns = list(macro_indicators_df.keys())
for col in macro_indicators_df.keys():
    merged[col] = merged[col].ffill()
    merged[col+"_pct_chng"] = merged[col].pct_change(periods=365)

# Make the data frame from 12/31/1999 to present
merged = merged[merged.index>=datetime(1999,12,31)]

# Write the csv to the current directory
merged.to_csv(os.getcwd() + "/macro_indicators.csv")

[*********************100%%**********************]  2 of 2 completed


<ipython-input-4-b929f5606dcf>:60: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Value_x'} in the result is deprecated and will raise a MergeError in a future version.
  merged = pd.merge(merged, macro_indicators_df[indicator], left_index=True, right_index=True, how='left')
<ipython-input-4-b929f5606dcf>:60: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Value_x'} in the result is deprecated and will raise a MergeError in a future version.
  merged = pd.merge(merged, macro_indicators_df[indicator], left_index=True, right_index=True, how='left')
<ipython-input-4-b929f5606dcf>:60: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Value_x'} in the result is deprecated and will raise a MergeError in a future version.
  merged = pd.merge(merged, macro_indicators_df[indicator], left_index=True, right_index=True, how='left')


In [5]:
# Pull the columns
macro_cols = list(macro_indicators_df.keys())

# Filter for only pct change columns
pct_cols = []
for col in macro_indicators_df.keys():
    if "2_YR" in col or "Fed_Funds" in col:
        pct_cols.append(col)
    else:
        pct_cols.append(col + '_pct_chng')

# Select specific columns
macro = merged[macro_cols]
pct = merged[pct_cols]

# Create correlation matrices
macro_corr = macro.corr()
pct_corr = pct.corr()

# Write correlation matrices to current directory
macro_corr.to_csv(os.getcwd() + "/macro_indicators_corr.csv")
pct_corr.to_csv(os.getcwd() + "/pct_chng_corr.csv")

In [7]:
# Make a copy of the macro data
micro_and_macro = merged.copy()
micro_and_macro = micro_and_macro[['Eff_Fed_Funds']]

# Read in company sentiment data
folder_path = os.path.join(os.getcwd(), 'company_sentiment_count')
dow_ciks = pd.read_excel("Dow_ciks.xlsx", sheet_name='Sheet1')
companies = list(dow_ciks['Name'])

company_sentiment_dfs = {}

for company in companies:
    cleaned_company = re.sub(r'[^a-zA-Z0-9\s]', '', company)
    company_name = cleaned_company.replace(' ', '_')
    file_name = company_name + '.csv'
    
    df = pd.read_csv(folder_path+'/'+file_name)
    
    col_name = company_name + '_sentiment_score'
    df[col_name] = df['sentiment_score']
    df.index = pd.to_datetime(df['date'])
    
    if "Goldman" in company or "JPMorgan" in company:
        continue
    
    micro_and_macro = pd.merge(micro_and_macro, df[col_name], left_index=True, right_index=True, how='left')
    micro_and_macro[col_name] = micro_and_macro[col_name].ffill()


cols = micro_and_macro.columns[1:]

micro_and_macro['total_sentiment'] = micro_and_macro[cols].sum(axis=1)
micro_and_macro['total_sentiment_count'] = micro_and_macro[cols].count(axis=1)

micro_and_macro['date'] = micro_and_macro.index
micro_and_macro.to_csv(os.getcwd() + "/micro_and_macro.csv")